In [74]:
class Computer:
    
    def __init__(self, program_string):
        
        self.t = []  # loading program makes this a list of ints
        self.load_program(program_string)
        self.current_pos = 0  # where we are reading memory from, start at 0
        self.func_dict = {}  # mapping of 2-digit opcode to function object
        self.args_dict = {}  # dict of how many args each function expects
        self.load_functions()
        self.input = []
        self.output = []  # queues of values written in or out
        self.gen = self.make_generator()  # the generator object that runs the code
        self.halted = False   # check from outside if program is complete
    
    def load_program(self, strin):
        
        """Loads in a comma delimited program string and sets up memory"""
        
        self.t = [int(x) for x in strin.split(",")]
        
    def load_functions(self):
        
        for x in dir(self):
            obj = getattr(self, x)
            doc = obj.__doc__
            if doc and len(doc) == 2:
                # it's an opcode function
                self.func_dict[doc] = obj
                if obj.__defaults__:
                    self.args_dict[doc] = len(obj.__defaults__)
                else:
                    self.args_dict[doc] = 0  # function expects no args
        # now we can look up functions by the opcode 2-character STRING

        
    ### Opcode function definitions, these are loaded by 
    ### introspection into a dict ###
    # a memory destination argument has a default of "dest" so that
    # the argument dispatcher can detect it's a memory address
    
    def add(self, a=None, b=None, dest="dest"):
        '''01'''
        self.t[dest] = a + b
        
    def multiply(self, a=None, b=None, dest="dest"):
        '''02'''
        self.t[dest] = a * b
        
    def get_input(self, dest="dest"):
        '''03'''
        # consumes one value from the input queue and writes it to memory
        if len(self.input) > 0:
            val = self.input.pop(0)  # FIFO queue
            self.t[dest] = val
        else:
            return "WAIT_FOR_INPUT"  # this causes the evaluation generator to yield
        
    def send_output(self, a=None):
        '''04'''
        # reads a value from memory and writes it to the output queue
        #print(f"outputting {self.t[dest]}")
        #self.output.append(self.t[a])
        self.output.append(a)
        
    def jump_if_true(self, a=None, b=None):   # changed b=dest to none
        '''05'''
        if not a == 0:
            self.current_pos = b
        else:
            self.current_pos += 3
            
    def jump_if_false(self, a=None, b=None):
        '''06'''
        if a == 0:
            self.current_pos = b
        else:
            self.current_pos += 3
            
    def less_than(self, a=None, b=None, dest="dest"):
        '''07'''
        if a < b:
            self.t[dest] = 1
        else:
            self.t[dest] = 0
            
    def equals(self, a=None, b=None, dest="dest"):
        '''08'''
        if a == b:
            self.t[dest] = 1
        else:
            self.t[dest] = 0
            
    def halt(self):
        '''99'''
        self.halted = True
        return "HALT"  # breaks out of the execution loop
    
    def prepare_args(self, func, modes, args):
        
        '''recieves a reference to func so it can determine whether
        its last argument is a memory address'''
        
        out = []
        if func.__defaults__:
            q = zip(modes, args, func.__defaults__)
            # an iterator to provide values in the loop
        else:
            return []  # func takes no arguments
        

        while True:
                
            try:
                m, a, default = next(q) # get the next mode and argument
            except StopIteration:  # never encountered a final argument "dest"
                break
            #print(m, a, default)
            if default == "dest":
                out.append(a)
                break  # the last argument is always a memory address to write to
                # zfill might make this appear as mode 0, but it's always
                # effectively mode 1
            if m == "0":  # positional mode
                out.append(self.t[a])
            elif m == "1":
                out.append(a)
        
        return out
    
    def make_generator(self):
        
        def gen():
            '''the actual object that runs the code'''
            
            while True:

                tmp = self.current_pos  # store to see if a jump happened, if so,
                # we don't need to advance the memory
                opcode = str(self.t[self.current_pos]).zfill(2)
                instruction = opcode[-2:]  # look up the instruction
                modes = opcode[:-2]  # up to and including third from the end
                required_args = self.args_dict[instruction]  # how many args
                
                modes = modes.zfill(required_args)
                modes = modes[::-1]
                arg_addrs = self.t[self.current_pos+1:self.current_pos+1+required_args]
                func = self.func_dict[instruction]
                #print(f"preparing args for {func} with modes {modes}")                
                args = self.prepare_args(func, modes, arg_addrs)
                
                # UNCOMMENT FOR DEBUGGING
                #print(f"Running {func.__name__} with args {args} in modes {modes}")
                
                z = func(*args)  # evaluate the function, check for return
                
                if z == "HALT":
                    #print("halting in while loop")
                    break  # opcode 99
                elif z == "WAIT_FOR_INPUT":
                    #print(f"Computer {self} is waiting for input")
                    yield
                    continue  # want to re-run the input instruction, not advance
                if tmp == self.current_pos:  # the current_pos has not been altered
                    self.current_pos += required_args + 1
                else:
                    pass
                    # a jump instruction has moved the current position
                    # so we don't need to advance it ourselves
                
            #print("before final while return statement")
            return
        
        return gen()
    
    def run(self):
        
        if self.halted:
            return False  # to send the signal that program is finished
        try:
            a = next(self.gen)
        except StopIteration:
            pass
            #print(f"Computer {self} has halted")
        if len(self.output) > 0:
            return self.output.pop()
        
    def send_input(self, value):
        
        self.input.append(value)

In [75]:
test1 = "3,12,6,12,15,1,13,14,13,4,13,99,-1,0,1,9"
test3 = "3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99"
day5 = "3,225,1,225,6,6,1100,1,238,225,104,0,1102,67,92,225,1101,14,84,225,1002,217,69,224,101,-5175,224,224,4,224,102,8,223,223,101,2,224,224,1,224,223,223,1,214,95,224,101,-127,224,224,4,224,102,8,223,223,101,3,224,224,1,223,224,223,1101,8,41,225,2,17,91,224,1001,224,-518,224,4,224,1002,223,8,223,101,2,224,224,1,223,224,223,1101,37,27,225,1101,61,11,225,101,44,66,224,101,-85,224,224,4,224,1002,223,8,223,101,6,224,224,1,224,223,223,1102,7,32,224,101,-224,224,224,4,224,102,8,223,223,1001,224,6,224,1,224,223,223,1001,14,82,224,101,-174,224,224,4,224,102,8,223,223,101,7,224,224,1,223,224,223,102,65,210,224,101,-5525,224,224,4,224,102,8,223,223,101,3,224,224,1,224,223,223,1101,81,9,224,101,-90,224,224,4,224,102,8,223,223,1001,224,3,224,1,224,223,223,1101,71,85,225,1102,61,66,225,1102,75,53,225,4,223,99,0,0,0,677,0,0,0,0,0,0,0,0,0,0,0,1105,0,99999,1105,227,247,1105,1,99999,1005,227,99999,1005,0,256,1105,1,99999,1106,227,99999,1106,0,265,1105,1,99999,1006,0,99999,1006,227,274,1105,1,99999,1105,1,280,1105,1,99999,1,225,225,225,1101,294,0,0,105,1,0,1105,1,99999,1106,0,300,1105,1,99999,1,225,225,225,1101,314,0,0,106,0,0,1105,1,99999,8,226,226,224,102,2,223,223,1005,224,329,1001,223,1,223,1108,677,677,224,1002,223,2,223,1006,224,344,101,1,223,223,1007,226,677,224,102,2,223,223,1005,224,359,101,1,223,223,1007,677,677,224,1002,223,2,223,1006,224,374,101,1,223,223,1108,677,226,224,1002,223,2,223,1005,224,389,1001,223,1,223,108,226,677,224,102,2,223,223,1006,224,404,101,1,223,223,1108,226,677,224,102,2,223,223,1005,224,419,101,1,223,223,1008,677,677,224,102,2,223,223,1005,224,434,101,1,223,223,7,677,226,224,1002,223,2,223,1005,224,449,101,1,223,223,1008,226,226,224,102,2,223,223,1005,224,464,1001,223,1,223,107,226,677,224,1002,223,2,223,1006,224,479,1001,223,1,223,107,677,677,224,102,2,223,223,1005,224,494,1001,223,1,223,1008,226,677,224,102,2,223,223,1006,224,509,1001,223,1,223,1107,677,226,224,102,2,223,223,1005,224,524,101,1,223,223,1007,226,226,224,1002,223,2,223,1006,224,539,1001,223,1,223,107,226,226,224,102,2,223,223,1006,224,554,101,1,223,223,108,677,677,224,1002,223,2,223,1006,224,569,1001,223,1,223,7,226,677,224,102,2,223,223,1006,224,584,1001,223,1,223,8,677,226,224,102,2,223,223,1005,224,599,101,1,223,223,1107,677,677,224,1002,223,2,223,1005,224,614,101,1,223,223,8,226,677,224,102,2,223,223,1005,224,629,1001,223,1,223,7,226,226,224,1002,223,2,223,1006,224,644,1001,223,1,223,108,226,226,224,1002,223,2,223,1006,224,659,101,1,223,223,1107,226,677,224,1002,223,2,223,1006,224,674,101,1,223,223,4,223,99,226"
day7 = "3,8,1001,8,10,8,105,1,0,0,21,46,67,76,101,118,199,280,361,442,99999,3,9,1002,9,4,9,1001,9,2,9,102,3,9,9,101,3,9,9,102,2,9,9,4,9,99,3,9,1001,9,3,9,102,2,9,9,1001,9,2,9,1002,9,3,9,4,9,99,3,9,101,3,9,9,4,9,99,3,9,1001,9,2,9,1002,9,5,9,101,5,9,9,1002,9,4,9,101,5,9,9,4,9,99,3,9,102,2,9,9,1001,9,5,9,102,2,9,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,99,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,99,3,9,1001,9,1,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1001,9,2,9,4,9,99,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,99"
testpu = "3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5"
# first phase, then input

In [78]:
from itertools import permutations, cycle
perm = permutations((5,6,7,8,9))
answers = []

for seq in perm:
    
    computers = []
    it = iter(seq)
    ainput = 0
    for _ in range(5):
        computers.append(Computer(day7))
    for c in computers:
        c.run()
        c.send_input(next(it))
        c.run()
        c.send_input(ainput)
        ainput = c.run()
    
    # now we have gone round the loop the first time
    
    it = cycle(computers)
    
    while True:
        last = ainput
        comp = next(it)
        comp.send_input(ainput)
        ainput = comp.run()
        if ainput is False:
            answers.append(last)
            break
    
    

In [79]:
max(answers)

17279674

In [72]:

computers = []
it = iter((9,8,7,6,5))
ainput = 0
for _ in range(5):
    computers.append(Computer(testpu))
for c in computers:
    c.run()
    c.send_input(next(it))
    c.run()
    c.send_input(ainput)
    ainput = c.run()
    
# now we have gone round the loop the first time
    
it = cycle(computers)
    
while True:
    last = ainput
    comp = next(it)
    comp.send_input(ainput)
    ainput = comp.run()
    if ainput is False:
        print(last)
        break
        
# 8726855

Computer <__main__.Computer object at 0x00000170480C7668> is waiting for input
Computer <__main__.Computer object at 0x00000170480C7668> is waiting for input
Computer <__main__.Computer object at 0x00000170480C7668> is waiting for input
Computer <__main__.Computer object at 0x00000170480C7DA0> is waiting for input
Computer <__main__.Computer object at 0x00000170480C7DA0> is waiting for input
Computer <__main__.Computer object at 0x00000170480C7DA0> is waiting for input
Computer <__main__.Computer object at 0x00000170480C7C18> is waiting for input
Computer <__main__.Computer object at 0x00000170480C7C18> is waiting for input
Computer <__main__.Computer object at 0x00000170480C7C18> is waiting for input
Computer <__main__.Computer object at 0x00000170480C7EF0> is waiting for input
Computer <__main__.Computer object at 0x00000170480C7EF0> is waiting for input
Computer <__main__.Computer object at 0x00000170480C7EF0> is waiting for input
Computer <__main__.Computer object at 0x00000170480C

In [62]:
a.run()

Computer <__main__.Computer object at 0x0000017048043DA0> has halted


999

In [64]:


computers = []
it = iter((9,8,7,6,5))
ainput = 0
for _ in range(5):
    computers.append(Computer(test1))
for c in computers:
    c.run()
    c.send_input(next(it))
    c.run()
    c.send_input(ainput)
    ainput = c.run()

False